# data preparation notebook

Start with importing all necessary libraries

In [45]:
import pandas as pd
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.stats import linregress
import folium
from scipy.stats import pearsonr
import streamlit as st
from streamlit_folium import folium_static
from pysal.lib import weights  
import segregation as seg
from sklearn.metrics import r2_score
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import shap


Read in all data

In [46]:
# Read zensus data in 100m x 100m grid
df1 = pd.read_csv('data/Zensus_Bremen_Buildings.csv', delimiter=';', index_col=0)
df2 = pd.read_csv('data/Zensus_Bremen_Families.csv', delimiter=';', index_col=0)
df3 = pd.read_csv('data/Zensus_Bremen_Households.csv', delimiter=';', index_col=0)
df4 = pd.read_csv('data/Zensus_Bremen_Population.csv', delimiter=';', index_col=0)
df5 = gpd.read_file('data/Zensus_Bremen_Grid_100m.gpkg')
df6 = gpd.read_file('data/Neighborhoods_Bremen.gpkg')
# Load the neighborhood polygons
neighborhood_polygons = gpd.read_file('data/Land_Prices_Neighborhood_Bremen.gpkg')

Now we transform the data to get one table to work with

In [47]:
# Merge them into a single dataframe by the column Grid_Code
merged_df = pd.merge(df1, df2, on='Grid_Code')
merged_df = pd.merge(merged_df, df3, on='Grid_Code')
merged_df = pd.merge(merged_df, df4, on='Grid_Code')
merged_df = pd.merge(merged_df, df5, on='Grid_Code')

zensus_data = gpd.GeoDataFrame(merged_df)
# Extract the grid middle points to match them later to their respective neighborhoods
zensus_data.geometry = zensus_data.geometry.centroid
# Reproject the data to use the same coordinate system
neighborhood_polygons = neighborhood_polygons.to_crs(zensus_data.crs)
# Join the grid points into the neighborhoods and group them
zensus_data_by_neighborhood = gpd.sjoin(zensus_data, neighborhood_polygons, how='left', op='within')
# Drop any non numeric data
zensus_data_by_neighborhood['BRW'] = pd.to_numeric(zensus_data_by_neighborhood.Land_Value, errors='coerce')
# Drop grid cells with not existing housing values
zensus_data_by_neighborhood = zensus_data_by_neighborhood.dropna(subset=['Land_Value'])
# Remove geometry column and add polygon areas
zensus_data_by_neighborhood = zensus_data_by_neighborhood.drop(columns=['geometry'])
zensus_data_by_neighborhood = pd.merge(zensus_data_by_neighborhood, neighborhood_polygons[['Neighborhood_FID', 'geometry']], on='Neighborhood_FID', how='left')
zensus_data_by_neighborhood_grouped = zensus_data_by_neighborhood.groupby('Neighborhood_FID')

# rename columns
column_names_map = {"Area_Types": "Area Type", "buildings_total_units": "Total number of buildings", "Land_Value": "Euros per square meter", "Neighborhood_FID": "Neighborhood ID", "geometry": "geometry", "sk_germany": "Amount of German citizens", "sk_abroad": "Amount of foreign citizens"}
age_group_map = {"alk_18_under": "Age Group under 18", "alk_18_29": "Age Group 18-29", "alk_30_49": "Age Group 30-49", "alk_50_64": "Age Group 50-64", "alk_65_over": "Age Group >65"}
building_age_map = {'j_before_1919': "Houses with building year under 1919",'j_1919_1948': "Houses with building year 1919-1948",'j_1949_1978': "Houses with building year 1948-1978",'j_1979_1995': "Houses with building year 1979-1995",'j_1996_2008': "Houses with building year 1996-2008",'j_2009_and_later': "Houses with building year >2009"}
household_map = {"hhgr_1_pers": "Household size 1", "hhgr_2_pers": "Household size 2", "hhgr_3_pers": "Household size 3", "hhgr_4_pers": "Household size 4", "hhgr_5_pers": "Household size 5", "hhgr_6_more": "Household size >=6"}
column_names_map.update(age_group_map)
column_names_map.update(building_age_map)
column_names_map.update(household_map)
zensus_data_by_neighborhood_filtered = zensus_data_by_neighborhood[column_names_map.keys()]
zensus_data_by_neighborhood_filtered = zensus_data_by_neighborhood_filtered.rename(columns=column_names_map)


In [48]:
agg_funcs = {column: 'sum' for column in column_names_map.values()}
agg_funcs.update({
    'Area Type': 'first',
    'geometry': 'first',
    'Euros per square meter': 'mean'
})
agg_funcs.pop('Neighborhood ID')

zensus_data_by_neighborhood = zensus_data_by_neighborhood_filtered.groupby("Neighborhood ID").agg(agg_funcs)

In [49]:
# Add Neighborhood name
df6 = df6.rename(columns={'Neighborhood_FID': 'Neighborhood ID', 'Neighborhood_Name': 'Neighborhood Name'})
df6 = df6[['Neighborhood ID', 'Neighborhood Name']]
zensus_data_by_neighborhood = pd.merge(zensus_data_by_neighborhood, df6, on='Neighborhood ID')

In [50]:
# Get percentage of German citizens
german = zensus_data_by_neighborhood["Amount of German citizens"]
foreign = zensus_data_by_neighborhood["Amount of foreign citizens"]
zensus_data_by_neighborhood["Percentage of German citizens"] = (german)/(german+foreign)

In [51]:
# Get number of residents per building
buildings = zensus_data_by_neighborhood["Total number of buildings"]
german = zensus_data_by_neighborhood["Amount of German citizens"]
foreign = zensus_data_by_neighborhood["Amount of foreign citizens"]
zensus_data_by_neighborhood["Amount of residents per building"] = (german+foreign)/buildings

In [52]:
# Normalize building data
buildings = zensus_data_by_neighborhood["Total number of buildings"]
for house_age in building_age_map.values():
    normalized = zensus_data_by_neighborhood[house_age]/buildings
    normalized.replace(np.nan, 0)
    zensus_data_by_neighborhood['Percentage of ' + house_age] = normalized
zensus_data_by_neighborhood=zensus_data_by_neighborhood.drop(columns=building_age_map.values())

In [53]:
# Load vegetation data and distance from center
df8 = pd.read_csv('data/bremen_district_spatial.csv', delimiter=',')
df8 = df8.rename(columns=dict(zip(df8.columns, ["Neighborhood ID", "Percentage of impervious land", "Percentage of water", "Percentage of vegetation", "Distance from center"])))
zensus_data_by_neighborhood = pd.merge(zensus_data_by_neighborhood, df8, on='Neighborhood ID')

In [54]:
# Normalize age groups
citizens = zensus_data_by_neighborhood["Amount of German citizens"] + zensus_data_by_neighborhood["Amount of foreign citizens"]
for age in age_group_map.values():
    normalized = zensus_data_by_neighborhood[age]/citizens
    normalized.replace(np.nan, 0)
    zensus_data_by_neighborhood['Percentage of ' + age] = normalized
zensus_data_by_neighborhood=zensus_data_by_neighborhood.drop(columns=age_group_map.values())

In [55]:
# Normalize household groups
households = zensus_data_by_neighborhood[household_map.values()].sum(axis=1)
for household in household_map.values():
    normalized = zensus_data_by_neighborhood[household]/households
    normalized.replace(np.nan, 0)
    zensus_data_by_neighborhood['Percentage of ' + household] = normalized
zensus_data_by_neighborhood=zensus_data_by_neighborhood.drop(columns=household_map.values())

In [56]:
zensus_data_by_neighborhood = zensus_data_by_neighborhood.drop(columns=["Amount of German citizens", "Amount of foreign citizens"])


In [57]:
gpd.GeoDataFrame(zensus_data_by_neighborhood).to_file('bremen_merged_data.gpkg', driver='GPKG')

In [58]:
X_data = zensus_data_by_neighborhood.drop(columns=['geometry', 'Area Type', 'Neighborhood Name'])
X_data.replace([np.inf, -np.inf], np.nan, inplace=True)
X_data.dropna()
y_data = X_data['Euros per square meter']
X_data.drop(columns=['Euros per square meter'], inplace=True)

# Define the XGBRegressor model
model = XGBRegressor()
model.fit(X_data, y_data)
# define model evaluation method
cv = RepeatedKFold(n_splits=4, n_repeats=3, random_state=13)
# evaluate model
scores = cross_val_score(model, X_data, y_data, scoring='r2', cv=cv, n_jobs=-1)
# force scores to be positive
scores = abs(scores)
print('r-squared: %.3f (%.3f)' % (scores.mean(), scores.std()) )

r-squared: 0.611 (0.111)


In [59]:
model.save_model("xgboost_model.json")